<a href="https://colab.research.google.com/github/gc2321/apache_beam/blob/main/3_side_input.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!{'pip install --quiet apache_beam'}

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.0/670.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup

In [2]:
import apache_beam as beam

## Side Input

In [3]:
# Mount Google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
import fileinput
deptData ="/content/gdrive/MyDrive/beam/dept_data.txt"
excludeData ="/content/gdrive/MyDrive/beam/exclude_ids.txt"

In [9]:
side_list = list()
with open(excludeData, 'r') as file:
  for line in file:
    side_list.append(line.rstrip())

class FilterUsingLength(beam.DoFn):
  def process(self, element, side_list, lower_bound, upper_bound=float('inf')):
    element_list = element.split(',')
    id = element_list[0]
    name = element_list[1]
    if (lower_bound <= len(name) <= upper_bound) and (id not in side_list):
       return [element_list]
    return []

p = beam.Pipeline()

small_name = (
    p
    | "Read from text file" >> beam.io.ReadFromText(deptData)
    | "ParDo with side input" >> beam.ParDo(FilterUsingLength(), side_list, 3 , 10)
    | beam.Map(lambda r: (r[0] + " " + r[1], 1))
    | beam.CombinePerKey(sum)
    | "Write result: " >> beam.io.WriteToText('data/output_new_final')
)

p.run()
!head -n 20 data/output_new_final-00000-of-00001

('503996WI Edouard', 31)
('957149WC Kyle', 31)
('241316NX Kumiko', 31)
('796656IE Gaston', 31)
('331593PS Beryl', 31)
('560447WH Olga', 31)
('222997TJ Leslie', 31)
('171752SY Mindy', 31)
('153636AS Vicky', 31)
('745411HT Richard', 31)
('298464HN Kirk', 31)
('783950BW Kaori', 31)
('892691AR Beryl', 31)
('245668UZ Oscar', 31)
('231206QD Kumiko', 31)
('357919KT Wendy', 31)
('472418ZG Cristobal', 31)
('442292OI Erika', 31)
('503647MN Sebastien', 31)
('245319LD Valerie', 31)
